[![Roboflow Notebooks](https://media.roboflow.com/notebooks/template/bannertest2-2.png?ik-sdk-version=javascript-1.4.3&updatedAt=1672932710194)](https://github.com/roboflow/notebooks)

# How to Train YOLOv8 Object Detection on a Custom Dataset

---

[![Roboflow](https://raw.githubusercontent.com/roboflow-ai/notebooks/main/assets/badges/roboflow-blogpost.svg)](https://blog.roboflow.com/how-to-train-yolov8-on-a-custom-dataset)
[![YouTube](https://badges.aleen42.com/src/youtube.svg)](https://youtu.be/wuZtUMEiKWY)
[![GitHub](https://badges.aleen42.com/src/github.svg)](https://github.com/ultralytics/ultralytics)

Ultralytics YOLOv8 is a popular version of the YOLO (You Only Look Once) object detection and image segmentation model developed by Ultralytics. The YOLOv8 model is designed to be fast, accurate, and easy to use, making it an excellent choice for a wide range of object detection and image segmentation tasks. It can be trained on large datasets and is capable of running on a variety of hardware platforms, from CPUs to GPUs.

## Disclaimer

If you notice that our notebook behaves incorrectly - especially if you experience errors that prevent you from going through the tutorial - don't hesitate! Let us know and open an [issue](https://github.com/roboflow/notebooks/issues) on the Roboflow Notebooks repository.

## Accompanying Blog Post

We recommend that you follow along in this notebook while reading the accompanying [Blog Post](https://blog.roboflow.com/how-to-train-yolov8-on-a-custom-dataset/).

## Pro Tip: Use GPU Acceleration

If you are running this notebook in Google Colab, navigate to `Edit` -> `Notebook settings` -> `Hardware accelerator`, set it to `GPU`, and then click `Save`. This will ensure your notebook uses a GPU, which will significantly speed up model training times.

## Steps in this Tutorial

In this tutorial, we are going to cover:

- Before you start
- Install YOLOv8
- CLI Basics
- Inference with Pre-trained COCO Model
- Roboflow Universe
- Preparing a custom dataset
- Custom Training
- Validate Custom Model
- Inference with Custom Model

**Let's begin!**

## Before you start

Let's make sure that we have access to GPU. We can use `nvidia-smi` command to do that. In case of any problems navigate to `Edit` -> `Notebook settings` -> `Hardware accelerator`, set it to `GPU`, and then click `Save`.

In [1]:
!nvidia-smi

Sat Nov 23 20:25:07 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   55C    P8              12W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [2]:
import os
HOME = os.getcwd()
print(HOME)

/content


## Install YOLOv8

YOLOv8 can be installed in two ways - from the source and via pip. This is because it is the first iteration of YOLO to have an official package.

In [3]:
# Pip install method (recommended)

!pip install ultralytics==8.2.103 -q

from IPython import display
display.clear_output()

import ultralytics
ultralytics.checks()

Ultralytics YOLOv8.2.103 🚀 Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
Setup complete ✅ (2 CPUs, 12.7 GB RAM, 32.5/112.6 GB disk)


In [4]:
from ultralytics import YOLO

from IPython.display import display, Image

## CLI Basics

If you want to train, validate or run inference on models and don't need to make any modifications to the code, using YOLO command line interface is the easiest way to get started. Read more about CLI in [Ultralytics YOLO Docs](https://docs.ultralytics.com/usage/cli/).

```
yolo task=detect    mode=train    model=yolov8n.yaml      args...
          classify       predict        yolov8n-cls.yaml  args...
          segment        val            yolov8n-seg.yaml  args...
                         export         yolov8n.pt        format=onnx  args...
```

## Inference with Pre-trained COCO Model

### 🐍 Python SDK

The simplest way of simply using YOLOv8 directly in a Python environment.

In [15]:
import cv2
import os
import numpy as np
import csv
#model = YOLO(f'{HOME}/yolov8n.pt')
#results = model.predict(source='https://media.roboflow.com/notebooks/examples/dog.jpeg', conf=0.25)
base_path = os.getcwd()


In [17]:
img_list = [[]for _  in range(1000)]
#合成video_path
video_path = ''
output_folder_path = os.path.join(base_path, 'output')
if not os.path.exists(output_folder_path):
  print(output_folder_path)
  os.makedirs(output_folder_path)

/content/output


In [9]:


cap = cv2.VideoCapture(video_path)
if not cap.isOpened():
  print(f"Error reading video file: {video_path}")
else:
#output_video的width, height, fps設定
  width, height, fps = (int(cap.get(x)) for x in (cv2.CAP_PROP_FRAME_WIDTH, cv2.CAP_PROP_FRAME_HEIGHT, cv2.CAP_PROP_FPS))
  #合成output_video_path
  if not os.path.exists(output_folder + "/videoOutput"):
      os.makedirs(output_folder + "/videoOutput")
    output_video_path = output_folder + "/videoOutput/"+filename[:-4]+"_output.mp4"

    video_writer = cv2.VideoWriter(output_video_path, cv2.VideoWriter_fourcc(*'mp4v'), fps, (width, height))
    #紀錄追蹤資訊
    track_info = defaultdict(lambda: {"frames": [], "bboxes": []})
    frame_num = 0
    #讀幀

    while cap.isOpened():
        success, frame = cap.read()
        if success:
            frame_num += 1
            #yoloV8啟動
            results = model.track(frame, persist=True, tracker='bytetrack.yaml', classes = [2,7])
            #要先確認影片中是否有偵測到物件
            if results[0].boxes.id is not None:
                #boxes為這一幀所有bounding box資訊（中心座標以及w,h)得集合
                #track_ids為這一幀所有id得集合

                boxes = results[0].boxes.xywh.cpu()
                track_ids = results[0].boxes.id.int().cpu().tolist()
                #將預測結果寫入影片（就是那些框框）
                annotated_frame = results[0].plot()
                cv2.putText(annotated_frame, str(frame_num), (20, 40), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 255))

                video_writer.write(annotated_frame)

                #將boxes和track_ids中的資料放入track_info
                for box, track_id in zip(boxes, track_ids):
                    x, y, w, h = box
                    track_info[track_id]["frames"].append(frame_num)
                    track_info[track_id]["bboxes"].append((x, y, w, h))
                    #計算bounding box左上角及右下角座標以供opencv截圖
                    x1 = int(x - w / 2)
                    y1 = int(y - h / 2)
                    x2 = int(x1 + w)
                    y2 = int(y1 + h)

                    if cut_with_outside == 1:
                        green_color = (0,255,0)
                        roi = np.ones((frame.shape[0],frame.shape[1],frame.shape[2]), np.uint8) * 255
                        roi[y1:y2, x1:x2] = frame[y1:y2, x1:x2]
                        if y1 != 0:
                            roi[y1-1, x1:x2] =  green_color
                        if x1 != 0:
                            roi[y1:y2, x1-1] =  green_color
                        if y2 != frame.shape[0] - 1 and y2 != frame.shape[0]:
                            roi[y2+1, x1:x2] =  green_color
                        if x2 != frame.shape[1] - 1 and x2 != frame.shape[1]:
                            roi[y1:y2, x2+1] =  green_color

                    elif cut_with_outside == 2 :
                        if x1 - 10 > 0:
                            x1 -= 10
                        else:
                            x1 = 0
                        if frame.shape[1] - x2 - 10 > 0:
                            x2 += 10
                        else:
                            x2 = frame.shape[1] - 1

                        roi = frame[y1:y2, x1:x2]
                        roi = cv2.resize(roi, (224, 224))
                    else:
                        roi = frame[y1:y2, x1:x2]
                    #合成img輸出path
                    img_list[track_id].append(roi)
            else:
                continue
        else:
            break
    model.predictor.trackers[0].reset()
    cap.release()
    video_writer.release()
    cv2.destroyAllWindows()
    #寫txt檔
    output_file = os.path.join(info_folder, f"{filename[:-4]}_track_info.txt")
    output_csvfile = os.path.join(info_folder, f"{filename[:-4]}_track_info.csv")
    with open(output_file, "w") as f:
        for track_id, info in track_info.items():
            frames = info["frames"]
            bboxes = info["bboxes"]
            f.write(f"Track ID: {track_id}\n")
            f.write("Bounding Boxes:\n")
            for bbox in bboxes:
                x, y, w, h = bbox
                f.write(f"    Frame: {frames[bboxes.index(bbox)]}, Bounding Box: ({x}, {y}, {w}, {h})\n")

    with open(output_csvfile, 'w', newline='') as csvoutput:
        writer = csv.writer(csvoutput)
        for track_id, info in track_info.items():
            frames = info["frames"]
            bboxes = info["bboxes"]
            for bbox in bboxes:
                x, y, w, h = bbox
                frameNum= frames[bboxes.index(bbox)]
                csvoutput.write(f'{track_id},{frameNum},{x:.17f},{y:.17f},{w},{h}\n')


# 新增區段